In [16]:
import os
import cv2
import numpy as np
import keras
from keras import layers, optimizers, models, regularizers
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [18]:
fixed_seed_num = 1234
np.random.seed(fixed_seed_num)
tf.random.set_seed(fixed_seed_num)

In [20]:
def generator_model(x_shape, y_shape):
    #encoder
    generator_input = layers.Input(batch_size=(None, x_shape, y_shape, 1), name="generator_input")
    generator_input_normalized = layers.BatchNormalization()(generator_input)

    conv1_32 = layers.Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(generator_input)
    conv1_32 = layers.BatchNormalization()(conv1_32)

    conv2_64 = layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv1_32)
    conv2_64 = layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv2_64)
    conv2_64 = layers.MaxPooling(pool_size=(2, 2), padding='same')(conv2_64)
    conv2_64 = layers.BatchNormalization()(conv2_64)

    conv3_128 = layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv2_64)
    conv3_128 = layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv3_128)
    conv3_128 = layers.MaxPooling(pool_size=(2, 2), padding='same')(conv3_128)
    conv3_128 = layers.BatchNormalization()(conv3_128)

    conv4_256 = layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv3_128)
    conv4_256 = layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv4_256)
    conv4_256 = layers.MaxPooling(pool_size=(2, 2), padding='same')(conv4_256)
    conv4_256 = layers.BatchNormalization()(conv4_256) 

    conv5_512 = layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv4_256)
    conv5_512 = layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv5_512)
    conv5_512 = layers.MaxPooling(pool_size=(2, 2), padding='same')(conv5_512)
    conv5_512 = layers.BatchNormalization()(conv5_512)

    conv6_512 = layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv5_512)
    conv6_512 = layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv6_512)
    conv6_512 = layers.MaxPooling(pool_size=(2, 2), padding='same')(conv6_512)
    conv6_512 = layers.BatchNormalization()(conv6_512)

    conv7_512 = layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='elu', kernel_regularizer=regularizers.l2(0.001))(conv7_512)
    conv7_512 = layers.BatchNormalization()(conv7_512)

    #decoder

    model = models.Model(inputs=generator_input, outputs=output)
    return model


In [22]:
def cGAN_model(generator, discriminator):
    discriminator.trainable = False
    model = models.Model(inputs=generator.inputs, outputs=[discriminator([generator.input, generator.output]), generator.output])
    return model